<a href="https://colab.research.google.com/github/nityanandkore/6162-KDD/blob/master/Homework4_Pattern_Mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf 6162-KDD
!git clone https://github.com/nityanandkore/6162-KDD.git

Cloning into '6162-KDD'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 84 (delta 34), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (84/84), done.


In [5]:
!pip install Orange3-Associate
import pandas 
import Orange
from Orange.data import Domain, DiscreteVariable, ContinuousVariable, Table
!pip install orange3
!pip install Orange3-Associate
from orangecontrib.associate.fpgrowth import *

ModuleNotFoundError: ignored

In [3]:
Titanic  = Orange.data.Table('6162-KDD/DataSet/Titanic.csv')

Titanic

NameError: ignored

In [0]:
#display all the variables in Titanic

Titanic.domain

[Class, Sex, Age | Survived]

In [0]:
#display all the predictor variables and their possible values
Titanic.domain.attributes

(DiscreteVariable(name='Class', values=['1st', '2nd', '3rd', 'Crew']),
 DiscreteVariable(name='Sex', values=['Female', 'Male']),
 DiscreteVariable(name='Age', values=['Adult', 'Child']))

In [0]:
#display the target variable and its possible values
#print(Titanic.domain.class_var)


Titanic.domain.class_var.name
print("Attributes:", ", ".join(x.name for x in Titanic.domain.attributes))
print("Class:", Titanic.domain.class_var.name)
print("Data instances", len(Titanic))

Attributes: Class, Sex, Age
Class: Survived
Data instances 2201


In [0]:
#display number of rows (or cases, tuples) in zoo
len(Titanic)


2201

In [0]:
#transform Titanic into a table X that has “one-hot” vector for each of the variables
from orangecontrib.associate.fpgrowth import *
X, mapping = OneHot.encode(Titanic, include_class=True)

X.shape


(2201, 10)

In [0]:
mapping.items()

dict_items([(0, (0, 0)), (1, (0, 1)), (2, (0, 2)), (3, (0, 3)), (4, (1, 0)), (5, (1, 1)), (6, (2, 0)), (7, (2, 1)), (8, (3, 0)), (9, (3, 1))])

In [0]:
#mine the frequent patterns with support threshold as 0.1
itemsets = dict(frequent_itemsets(X, .1))
len(itemsets)


35

In [0]:
list(itemsets)

[frozenset({0}),
 frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5}),
 frozenset({2, 5}),
 frozenset({3, 5}),
 frozenset({6}),
 frozenset({0, 6}),
 frozenset({1, 6}),
 frozenset({2, 6}),
 frozenset({3, 6}),
 frozenset({4, 6}),
 frozenset({5, 6}),
 frozenset({2, 5, 6}),
 frozenset({3, 5, 6}),
 frozenset({8}),
 frozenset({2, 8}),
 frozenset({3, 8}),
 frozenset({5, 8}),
 frozenset({2, 5, 8}),
 frozenset({3, 5, 8}),
 frozenset({6, 8}),
 frozenset({2, 6, 8}),
 frozenset({3, 6, 8}),
 frozenset({5, 6, 8}),
 frozenset({2, 5, 6, 8}),
 frozenset({3, 5, 6, 8}),
 frozenset({9}),
 frozenset({4, 9}),
 frozenset({5, 9}),
 frozenset({6, 9}),
 frozenset({4, 6, 9}),
 frozenset({5, 6, 9})]

In [0]:
#get the item names from the indices 
names = {item: '{}={}'.format(var.name, val)
          for item, var, val in OneHot.decode(mapping, Titanic, mapping)}

names


{0: 'Class=1st',
 1: 'Class=2nd',
 2: 'Class=3rd',
 3: 'Class=Crew',
 4: 'Sex=Female',
 5: 'Sex=Male',
 6: 'Age=Adult',
 7: 'Age=Child',
 8: 'Survived=No',
 9: 'Survived=Yes'}

In [0]:
#look at the transformed target variable indices
class_items = {item
                for item, var, _ in OneHot.decode(mapping, Titanic, mapping)
                if var is Titanic.domain.class_var}

class_items

{8, 9}

In [0]:
#generate rules on top of itemsets with confidence threshold 0.7
rules = [(P, Q, supp, conf)
          for P, Q, supp, conf in association_rules(itemsets, .7)
          if len(Q) == 1 and Q & class_items]

rules


[(frozenset({2, 5, 6}), frozenset({8}), 387, 0.8376623376623377),
 (frozenset({3, 5, 6}), frozenset({8}), 670, 0.777262180974478),
 (frozenset({2, 5}), frozenset({8}), 422, 0.8274509803921568),
 (frozenset({3, 5}), frozenset({8}), 670, 0.777262180974478),
 (frozenset({2, 6}), frozenset({8}), 476, 0.759170653907496),
 (frozenset({3, 6}), frozenset({8}), 673, 0.7604519774011299),
 (frozenset({5, 6}), frozenset({8}), 1329, 0.7972405518896221),
 (frozenset({4, 6}), frozenset({9}), 316, 0.7435294117647059),
 (frozenset({2}), frozenset({8}), 528, 0.7478753541076487),
 (frozenset({3}), frozenset({8}), 673, 0.7604519774011299),
 (frozenset({5}), frozenset({8}), 1364, 0.7879838243789717),
 (frozenset({4}), frozenset({9}), 344, 0.7319148936170212)]

In [0]:
#convert the indices in the rules to item names
for ante, cons, supp, conf in rules[:5]:
     print(', '.join(names[i] for i in ante), '-->',
           names[next(iter(cons))],
           '(supp: {}, conf: {})'.format(supp, conf))


Class=3rd, Sex=Male, Age=Adult --> Survived=No (supp: 387, conf: 0.8376623376623377)
Class=Crew, Sex=Male, Age=Adult --> Survived=No (supp: 670, conf: 0.777262180974478)
Class=3rd, Sex=Male --> Survived=No (supp: 422, conf: 0.8274509803921568)
Class=Crew, Sex=Male --> Survived=No (supp: 670, conf: 0.777262180974478)
Class=3rd, Age=Adult --> Survived=No (supp: 476, conf: 0.759170653907496)


In [0]:
!pip install pyfpgrowth
import pyfpgrowth

patterns = pyfpgrowth.find_frequent_patterns(Titanic, 0.1)

rules = pyfpgrowth.generate_association_rules(patterns, 0.7)

In [0]:
patterns

{(1.0, 1.0, 1.0, 2.0): 13,
 (0.0, 1.0, 1.0, 2.0): 124,
 (0.0, 1.0, 2.0): 1208,
 (0.0, 0.0, 1.0, 2.0): 480,
 (0.0, 2.0): 1351,
 (0.0, 0.0, 2.0): 747,
 (1.0, 1.0, 3.0): 192,
 (0.0, 1.0, 1.0, 3.0): 192,
 (0.0, 1.0, 3.0): 1764,
 (0.0, 0.0, 1.0, 3.0): 690,
 (0.0, 3.0): 1581,
 (0.0, 0.0, 3.0): 699,
 (1.0, 1.0, 1.0, 1.0): 11,
 (0.0, 1.0, 1.0, 1.0): 32,
 (0.0, 1.0, 1.0): 996,
 (0.0, 0.0, 1.0, 1.0): 292,
 (0.0, 1.0): 5049,
 (0.0, 0.0, 1.0): 2567,
 (0.0,): 4377,
 (0.0, 0.0): 2575}

In [0]:
rules

{(0.0, 2.0): ((1.0,), 0.8941524796447077),
 (1.0, 1.0, 3.0): ((0.0,), 1.0),
 (0.0, 3.0): ((1.0,), 1.1157495256166983),
 (0.0, 0.0, 3.0): ((1.0,), 0.9871244635193133),
 (0.0,): ((1.0,), 1.153529814941741),
 (0.0, 0.0): ((1.0,), 0.9968932038834951)}